In [1]:
import sys
sys.path.append("..")

In [2]:
import os
from models.segmentation_model import SegmentationModel
import tensorflow as tf
import numpy as np

# Define parametes

In [3]:
train_parameters = {
    "lr": 0.001,
    "decay_steps": 400,
    "decay_rate": 0.95,
    "dropout_prob": 0.05
}

model_parameters = {
    "speaker_embedding_size": 16,
    "num_conv_layers": 2,
    "conv_num_filters": 8,
    "conv_kernel_size": [2, 2],
    "num_bidirectional_units": 16,
    "num_bidirectional_layers": 2,
}

output_vocab_size= 50
num_speakers = 10
num_steps = 2
save_energy = 2
n_mels = 20
num_beams = 2

# Train model

In [4]:
with tf.Session() as sess:

    model = SegmentationModel(
        output_vocab_size, num_speakers, model_parameters,
    )
    
    frequencies = tf.placeholder(tf.float32, [None, None, n_mels])
    frequencies_seq_len = tf.placeholder(tf.int32, [None])
    speaker_ids = tf.placeholder(tf.int32, [None])
    phonemes = tf.sparse_placeholder(tf.int32)
    
    
    train_op_tf, loss_tf, global_step_tf, summary_tf = model.build_train_operations(
        frequencies, frequencies_seq_len,
        speaker_ids,
        phonemes,
        train_parameters
    )
    
    predictor_frequencies = tf.placeholder(tf.float32, [None, None, n_mels])
    predictor_frequencies_seq_len = tf.placeholder(tf.int32, [None])
    predictor_speaker_ids = tf.placeholder(tf.int32, [None])
    
    greedy_predictor_tf = model.build_greedy_predictor(
        predictor_frequencies, predictor_frequencies_seq_len, predictor_speaker_ids, True
    )
    
    beam_predictor_tf = model.build_beam_search_predictor(
        predictor_frequencies, predictor_frequencies_seq_len, predictor_speaker_ids,
        num_beams, True
    )

    train_writer = tf.summary.FileWriter('../log/train_grapheme_to_phoneme_model_notebook/train', sess.graph)
    
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    saver = tf.train.Saver(max_to_keep=3, keep_checkpoint_every_n_hours=3)
    
    for _ in xrange(num_steps):
        
        target_indices=[[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2]]
        target_values=[1, 2, 3, 4, 5, 6]
        dense_shape=[2, 4]
        
        out = sess.run([
            train_op_tf,
            global_step_tf,
            loss_tf,
            summary_tf
        ], feed_dict={
            frequencies: np.random.rand(2, 10, 20),
            frequencies_seq_len: 8* np.ones(2),
            speaker_ids: 2 * np.ones((2)),
            phonemes:(target_indices,target_values, dense_shape)
        })

        _, global_step, loss, summary  = out
        
        print global_step
        print loss

        train_writer.add_summary(summary, global_step)

        # detect gradient explosion
        if loss > 1e8 and global_step > 500:
            print('loss exploded')
            break

        if global_step % save_energy == 0 and global_step != 0:

            print('saving weights')
            if not os.path.exists('../weights/train_grapheme_to_phoneme_model_notebook/'):
                os.makedirs('../weights/train_grapheme_to_phoneme_model_notebook/')
            saver.save(sess, '../weights/train_grapheme_to_phoneme_model_notebook/', global_step=global_step)
    
    coord.request_stop()
    coord.join(threads)

InvalidArgumentError: Incompatible shapes: [2,10,20,8] vs. [2,8]
	 [[Node: encoder/conv_layers/conv_layer_1/mul = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](encoder/conv_layers/conv_layer_1/bn/batchnorm/add_1, encoder/dense/Softsign)]]

Caused by op u'encoder/conv_layers/conv_layer_1/mul', defined at:
  File "/Users/josebermeo/miniconda2/envs/deep_voice_2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/josebermeo/miniconda2/envs/deep_voice_2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/josebermeo/miniconda2/envs/deep_voice_2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/josebermeo/miniconda2/envs/deep_voice_2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/josebermeo/miniconda2/envs/deep_voice_2/lib/python2.7/s